In [1]:
from distgen import Generator
from distgen import PHYSICAL_CONSTANTS
from distgen.tools import check_abs_and_rel_tols

In [2]:
def test_getitem_():
    ins = """
n_particle: 200000
species: electron
r_dist:
  sigma_xy: 1.0 mm
  type: radial_gaussian
random:
  type: hammersley
start:
  tstart: 0 sec
  type: time
total_charge: 10 pC
"""

    D = Generator(ins)
    assert D["r_dist:sigma_xy"]["value"] == 1
    assert D["r_dist:sigma_xy"]["units"] == "millimeter"


test_getitem_()

In [3]:
def test_setitem_():
    ins = """
n_particle: 200000
species: electron
r_dist:
  sigma_xy: 1.0 mm
  type: radial_gaussian
random:
  type: hammersley
start:
  tstart: 0 sec
  type: time
total_charge: 10 pC
"""

    D = Generator(ins)
    D["r_dist:sigma_xy"] = 2.0

    assert isinstance(D["r_dist:sigma_xy"], dict)
    assert D["r_dist:sigma_xy"]["value"] == 2
    assert D["r_dist:sigma_xy"]["units"] == "millimeter"

    D["r_dist:sigma_xy:units"] = "cm"
    assert D["r_dist:sigma_xy"]["value"] == 2
    assert D["r_dist:sigma_xy"]["units"] == "centimeter"

    D["r_dist:sigma_xy:value"] = 3
    assert D["r_dist:sigma_xy"]["value"] == 3
    assert D["r_dist:sigma_xy"]["units"] == "centimeter"

    D["r_dist:sigma_xy"] = "4 um"
    assert D["r_dist:sigma_xy"]["value"] == 4
    assert D["r_dist:sigma_xy"]["units"] == "micrometer"


test_setitem_()

In [4]:
def test_species_rest_energy():
    inputs = {
        "n_particle": 1000,
        "total_charge": "0 C",
        "species": None,
        "start": {"type": "time"},
    }

    for species in PHYSICAL_CONSTANTS.species_list:
        inputs["species"] = species
        inputs["total_charge"] = (
            f'{inputs["n_particle"]*PHYSICAL_CONSTANTS.species(species)["charge"].magnitude} C'
        )

        D = Generator(inputs)
        B = D.beam()

        check_abs_and_rel_tols(
            "rest energy",
            B["energy"],
            PHYSICAL_CONSTANTS.species(species)["mc2"],
            abs_tol=1e-6,
        )


test_species_rest_energy()